In [19]:
import math
import copy
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [11]:
def compute_guidance(state, t, t_index):
    
    if not hasattr(compute_guidance, "counter"):
         compute_guidance.counter = 0
         compute_guidance.cmd0 = 0
         compute_guidance.cmd1 = 0
         compute_guidance.cmd2 = 0
         compute_guidance.cmd3 = 0
            
            
    u,v,w,p,q,r,x,y,z,phi,the,psi = state[:12]  # state variables(u,v,w : 기체중심 좌표 / p,q,r : 각속도 / x,y,z : NED 기준 좌표
                                                #                phi,the,psi : 자세각 )
    
    s_phi, c_phi = np.sin(phi), np.cos(phi)
    s_the, c_the = np.sin(the), np.cos(the)
    s_psi, c_psi = np.sin(psi), np.cos(psi)
    
    Cbn = np.array([ \
        [c_psi*c_the, c_psi*s_the*s_phi-s_psi*c_phi, c_psi*s_the*c_phi+s_psi*s_phi], \
        [s_psi*c_the, s_psi*s_the*s_phi+c_psi*c_phi, s_psi*s_the*c_phi-c_psi*s_phi], \
        [     -s_the,                   c_the*s_phi,                   c_the*c_phi] \
    ])

    x_dot,y_dot,z_dot = np.dot(Cbn,[u,v,w])
    h_dot = -z_dot
    h = -z
    
    omg_alt, zet_alt = 2, 0.8
    omg_pos, zet_pos = 2, 0.8
    
    # NED 좌표 기준 위치 명령 h, x, y와 자세각 명령 psi
    if t < t_fail+t_detect:
        
        # 고장 전 자세 명령 산출 loop
        
        h_cmd = 1
        x_cmd = 1
        y_cmd = 2
        psi_cmd = 10*Deg2Rad
    
        # altitude loop
        up_cmd = -2*zet_alt*omg_alt*h_dot + omg_alt**2*(h_cmd-h) + g
        thrust = m*up_cmd/(c_phi*c_the)

        # horizontal position loop
        ax_cmd = -2*zet_pos*omg_pos*x_dot + omg_pos**2*(x_cmd-x)
        ay_cmd = -2*zet_pos*omg_pos*y_dot + omg_pos**2*(y_cmd-y)
        au_cmd, av_cmd, aw_cmd = np.dot(Cbn.T,[ax_cmd,ay_cmd,0])
        the_cmd = np.arctan2(-au_cmd,g)
        phi_cmd = np.arctan2( av_cmd,g)
    
    else :
        
        # 고장 검출 후 자세 명령 산출 loop(compute_guidance_cvx함수로 계산)
        
        if (compute_guidance.counter%5==0):
            thrust, phi_cmd, the_cmd, psi_cmd = compute_guidance_cvx([x,y,z],[x_dot,y_dot,z_dot],psi,tf-t)
            compute_guidance.cmd0 = thrust
            compute_guidance.cmd1 = phi_cmd
            compute_guidance.cmd2 = the_cmd
            compute_guidance.cmd3 = psi_cmd
            
        else:
            thrust = compute_guidance.cmd0
            phi_cmd = compute_guidance.cmd1
            the_cmd = compute_guidance.cmd2
            psi_cmd = compute_guidance.cmd3
        
        compute_guidance.counter += 1
        
    return np.array([thrust, phi_cmd, the_cmd, psi_cmd])

In [9]:
# guidance command computation_convex
def  compute_guidance_cvx(position, velocity, psi, tgo):
    delt = 0.05
    T = tgo
    N = int(T/delt)

    r_des = np.zeros(3)
    r_des[0] = 1
    r_des[1] = 1
    r_des[2] = 0

    v_des = np.zeros(3)

    g_pin = np.zeros(3)
    g_pin[2] = 9.8
    T2 = 720

    T_c = cp.Variable((3,N+1))
    r_pin = cp.Variable((3,N+1))
    v_pin = cp.Variable((3,N+1))

    mag = cp.norm(T_c,axis=0)
    obj = cp.Minimize(cp.norm(mag))    

    constr = [ 
        v_pin[:,0] == velocity,
        r_pin[:,0] == position,
        v_pin[:,-1] == v_des,
        r_pin[:,-1] == r_des
        ]
    
    # 3축 최적 추력 명령 산출
    
    for t in range(N):
        constr += [        
            r_pin[:,t+1] == r_pin[:,t] + v_pin[:,t]*delt,
            v_pin[:,t+1] == v_pin[:,t] + (T_c[:,t]/m+g_pin)*delt,
            cp.norm(T_c[:,t]) <= T2,
            T_c[2,t]<=0,
        ]

    prob = cp.Problem(obj, constr)
    prob.solve(verbose=False,solver=cp.ECOS)

    if prob.status=='optimal' or prob.status=='optimal_inaccurate':
        T_n, T_e, T_d = T_c.value[:,0]
    
    # 추력 명령 -> 자세각 명령 변환 loop
    
        T_a =  T_n*np.cos(psi) + T_e*np.sin(psi)
        T_b = -T_n*np.sin(psi) + T_e*np.cos(psi)
    
        thrust = np.sqrt(T_d**2+T_n**2+T_e**2)
        the_cmd = -np.arctan2(T_a,-T_d)
        phi_cmd = np.arcsin(T_b/thrust)
        psi_cmd = psi
        print(tgo, prob.status)
       
    else:
        thrust, phi_cmd, the_cmd, psi_cmd = 0, 0, 0, 0
        print(tgo, prob.status)
    
    if position[2]>=0:
        thrust, phi_cmd, the_cmd, psi_cmd = 0, 0, 0, 0 

    return np.array([thrust,phi_cmd,the_cmd,psi_cmd])

In [21]:

dt = 0.01            # control frequency
tf = 7             # final time
g = 9.8              # m/s^2
t_fail = 3         # 고장 시간
t_detect = 0.2     # 고장 검출에 걸리는 시간  
t = np.arange(0,tf, dt)
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad
m = 23.56            # kg
test = np.zeros(12)

start_time = time.time()
compute_guidance(test,t[0],1)
end_time = time.time()

elapsed_time = end_time - start_time

print(elapsed_time)

0.0002148151397705078


In [10]:
def compute_attitude_angle(init_pos,init_vel,target_landing_point):
    pos_x = init_pos[0]
    pos_y = init_pos[1]
    pos_z = init_pos[2]
    
    vel_x = init_vel[0]
    vel_y = init_vel[1]
    vel_z = init_vel[2]

    target_x = target_landing_pointp[0] #desire_landing point
    target_y = target_landing_pointp[1] #desire_landing point
    target_z = target_landing_pointp[2] #maybe 0?

    #####################computing stage######################

    thrust, phi_roll, theta_pitch, psi_ty = compute_guidance()
    
    ##########################################################
    return roll,pitch,yaw,thrust